<a href="https://colab.research.google.com/github/Hanalia/Colab_Examples/blob/main/openai_embeddings_textgrouping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI Embeddings API를 활용하여 유사한 텍스트끼리 묶기


OpenAI에서는 Embeddings API를 제공하고 있어서 매우 저렴한 비용으로 빠르게 텍스트 임베딩을 할 수 있다.[링크](https://platform.openai.com/docs/guides/embeddings)


## 필요 라이브러리 설치

In [3]:
## 필요 라이브러리 설치
!pip install pandas
!pip install openai
!pip install scikit-learn
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import Libraries

In [4]:
import pandas as pd
import openai
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from datetime import datetime

## 핵심 모듈 (EmbeddingProcessor) 선언

In [36]:

class EmbeddingProcessor:
    def __init__(self, api_key):
        openai.api_key = api_key
        self.headers = {'Authorization': 'Bearer ' + api_key}

    def get_embedding(self, text, model="text-embedding-ada-002"):
        try:
            text = text.replace("\n", " ")
            return openai.Embedding.create(input=[text], model=model)['data'][0]['embedding']
        except Exception as e:
            print(f"Error getting embedding: {e}")
            return None
    
    ## for local files
    def load_and_prepare_data(self, input_file_name, column_name):
        try:
            if input_file_name.endswith('.csv'):
                df = pd.read_csv(input_file_name)
            elif input_file_name.endswith('.xlsx'):
                df = pd.read_excel(input_file_name)
            else:
                print("Unsupported file type. Please provide a .csv or .xlsx file.")
                return None

            df['np_embedding'] = df[column_name].apply(lambda x: np.array(self.get_embedding(x, model='text-embedding-ada-002')))
            return df
        except Exception as e:
            print(f"Error loading and preparing data: {e}")
            return None

    ## for local files
    def prepare_data(self, input_df, column_name):
        try:

            df['np_embedding'] = df[column_name].apply(lambda x: np.array(self.get_embedding(x, model='text-embedding-ada-002')))
            return df
        except Exception as e:
            print(f"Error loading and preparing data: {e}")
            return None


    def process_data(self, df, threshold=0.95, output_format='xlsx'):
        try:
            similarity_matrix = cosine_similarity(np.stack(df['np_embedding'].values))
            groups = []
            for i in range(len(df)):
                if any(i in group for group in groups):
                    continue
                similar_articles = np.where(similarity_matrix[i] > threshold)[0]
                groups.append(similar_articles)
            df['group'] = -1
            for group_label, group in enumerate(groups):
                df.loc[group, 'group'] = group_label
            df = df.drop(columns=['np_embedding'])  # Drop the np_embedding column from the final dataframe
            return df
        except Exception as e:
            print(f"Error processing and saving data: {e}")
            return None


## OpenAI API Key 입력하기

In [6]:
OPENAI_API_KEY = "" #@param {type:"string"}

## 텍스트가 담긴 파일 (.xlsx 또는 .csv) 업로드하기

In [48]:
import io
from google.colab import files
uploaded = files.upload()
file_path = list(uploaded.keys())[0]


Saving embeddings_예시데이터.xlsx to embeddings_예시데이터.xlsx


##클래스 인스턴스 실행 후 각 행에 대한 embeddings 생성

In [49]:
processor = EmbeddingProcessor(api_key=OPENAI_API_KEY)
df = processor.load_and_prepare_data(file_path,"내용")
df

,링크,제목,내용,np_embedding
0,https://www.news1.kr/articles/5072006,"과기정통부, 베트남에서 인공지능·디지털 전환 협력 포럼 개최",과학기술정보통신부는 베트남 과학기술부와 함께 베트남 하노이 롯데호텔에서 '한-베트남...,"[-0.005398707929998636, -0.0031290266197174788..."
1,https://www.yna.co.kr/view/AKR2023060904730001...,"한국·베트남 정부, 의료 인공지능·디지털전환 포럼",(서울=연합뉴스) 오규진 기자 = 과학기술정보통신부는 베트남 하노이에서 베트남 과학...,"[0.00264614075422287, -0.009957978501915932, 0..."
2,https://www.busan.com/view/busan/view.php?code...,"\n생성형 인공지능에 밀렸던 메타버스, 다시 살아난다","생성형 인공지능(AI) 챗GPT, 바드의 등장으로 한 때 시들해졌던 메타버스 생태계...","[-0.004142578691244125, -0.017906852066516876,..."
3,http://www.dtnews24.com/news/articleView.html?...,"유성구, AI(인공지능) 케어콜 본격 추진",대전 유성구(구청장 정용래)는 9일 네이버클라우드㈜와 업무협약을 체결하고 ‘AI(인...,"[-0.0017294853460043669, -0.019334932789206505..."
4,https://news.jtbc.co.kr/article/article.aspx?n...,핵심산업으로 떠오른 인공지능…갈 길 바쁜 '한국판 챗GPT',"[앵커]\n\n인공지능 시장이 커지자 나라 간, 기업 간의 주도권 다툼도 치열합니다...","[-0.0051900167018175125, -0.021334508433938026..."
5,https://www.dnews.co.kr/uhtml/view.jsp?idxno=2...,"경기도, 인공지능(AI) 전문가 정책 간담회 개최","일리야 폴로수킨, 인공지능 기술과 공공적용 사례 발표\n\n국내 전문가와 함께 ‘경...","[0.0047479779459536076, -0.025937218219041824,..."
6,http://www.enewstoday.co.kr/news/articleView.h...,"경기도, ‘인공지능(AI) 전문가 정책 간담회’ 열어",[이뉴스투데이 경기1취재본부 김승희 기자] 김동연 경기도지사는 8일 도담소에서 진행...,"[-0.0043725548312067986, -0.01998882181942463,..."
7,https://www.asiatime.co.kr/article/20230609500035,경기도 '인공지능 전문가 정책간담회' 개최,[아시아타임즈 경기취재본부=임덕철 기자] 경기도는 지난 8일 도담소에서 일리야 폴로...,"[0.0034814036916941404, -0.021934855729341507,..."


## Text Grouping
threshold : 0 ~ 1 사이 숫자
1에 가까워질 수록 완전히 일치하는 텍스트만 같은 그룹으로 분류됨

### CASE 1 : Threshold = 0.89 : 경기도 정책 간담회 텍스트는 제대로 Grouping 되지 않음

In [50]:

threshold = 0.89
df1 = processor.process_data(df,threshold)
df1


,링크,제목,내용,group
0,https://www.news1.kr/articles/5072006,"과기정통부, 베트남에서 인공지능·디지털 전환 협력 포럼 개최",과학기술정보통신부는 베트남 과학기술부와 함께 베트남 하노이 롯데호텔에서 '한-베트남...,0
1,https://www.yna.co.kr/view/AKR2023060904730001...,"한국·베트남 정부, 의료 인공지능·디지털전환 포럼",(서울=연합뉴스) 오규진 기자 = 과학기술정보통신부는 베트남 하노이에서 베트남 과학...,0
2,https://www.busan.com/view/busan/view.php?code...,"\n생성형 인공지능에 밀렸던 메타버스, 다시 살아난다","생성형 인공지능(AI) 챗GPT, 바드의 등장으로 한 때 시들해졌던 메타버스 생태계...",1
3,http://www.dtnews24.com/news/articleView.html?...,"유성구, AI(인공지능) 케어콜 본격 추진",대전 유성구(구청장 정용래)는 9일 네이버클라우드㈜와 업무협약을 체결하고 ‘AI(인...,2
4,https://news.jtbc.co.kr/article/article.aspx?n...,핵심산업으로 떠오른 인공지능…갈 길 바쁜 '한국판 챗GPT',"[앵커]\n\n인공지능 시장이 커지자 나라 간, 기업 간의 주도권 다툼도 치열합니다...",2
5,https://www.dnews.co.kr/uhtml/view.jsp?idxno=2...,"경기도, 인공지능(AI) 전문가 정책 간담회 개최","일리야 폴로수킨, 인공지능 기술과 공공적용 사례 발표\n\n국내 전문가와 함께 ‘경...",2
6,http://www.enewstoday.co.kr/news/articleView.h...,"경기도, ‘인공지능(AI) 전문가 정책 간담회’ 열어",[이뉴스투데이 경기1취재본부 김승희 기자] 김동연 경기도지사는 8일 도담소에서 진행...,2
7,https://www.asiatime.co.kr/article/20230609500035,경기도 '인공지능 전문가 정책간담회' 개최,[아시아타임즈 경기취재본부=임덕철 기자] 경기도는 지난 8일 도담소에서 일리야 폴로...,2


### CASE 2 : Threshold = 0.95 : 의도한 대로 텍스트가 Grouping 됨

In [51]:
threshold = 0.95
df2 = processor.process_data(df,threshold)
df2


,링크,제목,내용,group
0,https://www.news1.kr/articles/5072006,"과기정통부, 베트남에서 인공지능·디지털 전환 협력 포럼 개최",과학기술정보통신부는 베트남 과학기술부와 함께 베트남 하노이 롯데호텔에서 '한-베트남...,0
1,https://www.yna.co.kr/view/AKR2023060904730001...,"한국·베트남 정부, 의료 인공지능·디지털전환 포럼",(서울=연합뉴스) 오규진 기자 = 과학기술정보통신부는 베트남 하노이에서 베트남 과학...,0
2,https://www.busan.com/view/busan/view.php?code...,"\n생성형 인공지능에 밀렸던 메타버스, 다시 살아난다","생성형 인공지능(AI) 챗GPT, 바드의 등장으로 한 때 시들해졌던 메타버스 생태계...",1
3,http://www.dtnews24.com/news/articleView.html?...,"유성구, AI(인공지능) 케어콜 본격 추진",대전 유성구(구청장 정용래)는 9일 네이버클라우드㈜와 업무협약을 체결하고 ‘AI(인...,2
4,https://news.jtbc.co.kr/article/article.aspx?n...,핵심산업으로 떠오른 인공지능…갈 길 바쁜 '한국판 챗GPT',"[앵커]\n\n인공지능 시장이 커지자 나라 간, 기업 간의 주도권 다툼도 치열합니다...",3
5,https://www.dnews.co.kr/uhtml/view.jsp?idxno=2...,"경기도, 인공지능(AI) 전문가 정책 간담회 개최","일리야 폴로수킨, 인공지능 기술과 공공적용 사례 발표\n\n국내 전문가와 함께 ‘경...",4
6,http://www.enewstoday.co.kr/news/articleView.h...,"경기도, ‘인공지능(AI) 전문가 정책 간담회’ 열어",[이뉴스투데이 경기1취재본부 김승희 기자] 김동연 경기도지사는 8일 도담소에서 진행...,4
7,https://www.asiatime.co.kr/article/20230609500035,경기도 '인공지능 전문가 정책간담회' 개최,[아시아타임즈 경기취재본부=임덕철 기자] 경기도는 지난 8일 도담소에서 일리야 폴로...,4


### 데이터프레임 저장하기

In [52]:
df2.to_csv('output.csv', encoding = 'utf-8-sig') 
files.download('output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>